<a href="https://colab.research.google.com/github/Seydifa/Challenge/blob/main/CGIAR_Root_Volume_Estimation_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

papitosen_cgiar_root_volume_estimation_challenge_dataset_path = kagglehub.dataset_download('papitosen/cgiar-root-volume-estimation-challenge-dataset')

print('Data source import complete.')

100%|██████████| 643M/643M [00:08<00:00, 81.9MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install timm ultralytics -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from sklearn.model_selection import train_test_split
from torchvision import transforms
from tqdm.notebook import tqdm
from ultralytics import YOLO
from PIL import Image
from glob import glob
import pandas as pd
import numpy as np
import torch
import timm
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
os.listdir(papitosen_cgiar_root_volume_estimation_challenge_dataset_path)

['data', 'Models', 'Test (1).csv', 'Train (2).csv']

In [ ]:
DATA_PATH = papitosen_cgiar_root_volume_estimation_challenge_dataset_path + '/data'
print(os.listdir(DATA_PATH))
TRAIN_DATA = DATA_PATH + '/train'
TEST_DATA = DATA_PATH + '/test'

print('Numbers of files in train', len(os.listdir(TRAIN_DATA)))
print('Numbers of files in test', len(os.listdir(TEST_DATA)))

['test', 'train']
Numbers of files in train 98
Numbers of files in test 62


In [ ]:
os.listdir(papitosen_cgiar_root_volume_estimation_challenge_dataset_path + '/Models' + '/Models')

['best_early.pt', 'best_full.pt', 'best_late.pt']

In [ ]:
MODEL_PATHS = glob(papitosen_cgiar_root_volume_estimation_challenge_dataset_path + '/Models/Models/*')
MODE_NAMES = ['Early', 'Full', 'Late']
MODEL_LISTS = []
for name, path in zip(MODE_NAMES, MODEL_PATHS):
    model = YOLO(path)
    MODEL_LISTS.append((name, model))

MODEL_LISTS = dict(MODEL_LISTS)

In [ ]:
MODEL_LISTS.keys()

dict_keys(['Early', 'Full', 'Late'])

In [ ]:
DETECTION_LIST = ['Early', 'Late']
DETECTION_MODELS = {
    'Early': [MODEL_LISTS['Full']],
    'Late': [MODEL_LISTS['Full']]
}

for detection in DETECTION_LIST:
  DETECTION_MODELS[detection].append(MODEL_LISTS[detection])
DETECTION_MODELS.keys()

dict_keys(['Early', 'Late'])

In [ ]:
TRAIN_CSV = papitosen_cgiar_root_volume_estimation_challenge_dataset_path + '/Train (2).csv'
TEST_CSV = papitosen_cgiar_root_volume_estimation_challenge_dataset_path + '/Test (1).csv'

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)

Train shape: (386, 9)
Test shape: (130, 8)


In [ ]:
train_df

,ID,FolderName,PlantNumber,Side,Start,End,RootVolume,Genotype,Stage
0,ID_826322_Lbmaya,A6dzrkjqvl,3,L,33,42,0.9,IITA-TMS-IBA000070,Early
1,ID_718181_Bslpwx,Ypktwvqjbn,7,L,33,41,1.5,IBA154810,Early
2,ID_465762_L1n61d,Ox18ob0syv,4,R,21,28,2.7,IBA980581,Early
3,ID_626872_Pbmx2e,Hqcekwpxgu,2,R,30,39,2.6,IITA-TMS-IBA000070,Early
4,ID_518846_Opko8c,Ummqfuetoc,1,R,17,26,2.7,IBA980581,Early
...,...,...,...,...,...,...,...,...,...
381,ID_592215_Hfzzen,Ntqusw7xpb,4,L,19,31,2.2,TMEB419,Early
382,ID_995502_L5bkz9,Izbgyxre0g,4,R,22,33,1.3,IBA154810,Early
383,ID_490708_Isn2o5,Efaiye0hmy,1,R,20,30,4.8,IBA980581,Early
384,ID_899134_Gyrlfg,Rzyf8hal4j,1,R,38,47,1.3,IBA154810,Early


In [ ]:
sample = train_df.sample(1).to_dict('records')[0]
sample

{'ID': 'ID_619512_Hveydf',
 'FolderName': 'Dz9lpsl9ae',
 'PlantNumber': 2,
 'Side': 'R',
 'Start': 45,
 'End': 50,
 'RootVolume': 0.9,
 'Genotype': 'TMEB419',
 'Stage': 'Early'}

In [ ]:
#print("Features extractors: ", [name for name in timm.list_models(pretrained=True)  if  'resnet' in name])
#model_name = 'resnet18'

#feature_depth = -1
#feature_extractor = timm.create_model(model_name, pretrained=True, features_only=True, num_classes=0)
#feature_extractor.eval()

#model_config = timm.data.resolve_data_config(feature_extractor.pretrained_cfg)
#model_config['input_size'] = (3, 64, 128)
#transform = timm.data.create_transform(**model_config)
#print("feature dim", feature_extractor.feature_info.channels()[feature_depth])

def prepare_image(image):
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)
    gray = image.convert('L')
    gray = np.asarray(gray)
    gray = np.tile(gray[..., None], (1, 1, 3))
    gray = Image.fromarray(gray)
    gray = transform(gray)
    return gray

@torch.inference_mode()
def extract_feature(image_cropped):
    image_cropped = [prepare_image(image) for image in image_cropped]
    image_cropped = torch.stack(image_cropped)
    feature = feature_extractor(image_cropped)[feature_depth]
    feature = feature.mean(dim=(2, 3))
    return feature

In [ ]:
train_df.describe()

,PlantNumber,Start,End,RootVolume
count,386.000000,386.000000,386.000000,386.000000
mean,3.974093,27.018135,36.989637,2.052306
std,1.508195,8.234686,7.960611,1.526302
min,1.000000,8.000000,14.000000,0.000000
25%,3.000000,22.000000,32.000000,0.900000
50%,4.000000,26.000000,37.000000,1.900000
75%,5.000000,32.000000,42.000000,2.700000
max,7.000000,49.000000,60.000000,11.000000


In [ ]:
START_BEGIN, END_BEGIN = 7, 60
DEPTH = END_BEGIN - START_BEGIN
VOLUME_WIDTH, VOLUME_HEIGHT = 512, 64

def load_images(image_dir, folder_name, side, start=None, end=None):
    """
    Load images from a directory and return them along with their depths.

    Args:
        image_dir (str): The directory containing the images.
        folder_name (str): The folder within the directory containing the images.
        side (str): The side of the images to load (e.g., 'left', 'right').
        start (int, optional): The starting index of the images to load.
        end (int, optional): The ending index of the images to load.

    Returns:
        tuple: A tuple containing a list of images and a list of depths.
    """
    filepaths = sorted(glob(os.path.join(image_dir, folder_name, f"*_{side}_*.png")))
    if start is not None and end is not None:
        filepaths = filepaths[start:end]

    images = [Image.open(filepath) for filepath in filepaths]
    depths = [int(filepath.split('_')[-1].split('.')[0]) for filepath in filepaths]

    return images, depths

@torch.inference_mode()
def detect_image(images, depths, stage, plant_idx, verbose=False):
    """
    Detect and crop regions of interest from images based on detection models.

    Args:
        images (list): List of PIL images.
        depths (list): List of depths corresponding to the images.
        stage (str): The stage of the detection model to use.
        plant_idx (int): The index of the plant to detect.
        verbose (bool, optional): Whether to print debug information.

    Returns:
        np.ndarray: A 4D numpy array representing the volumes of cropped images.
    """
    full_model = DETECTION_MODELS[stage][0]
    stage_model = DETECTION_MODELS[stage][1]

    full_results = full_model(images, verbose=False)
    stage_results = stage_model(images, verbose=False)

    volumes = np.zeros((DEPTH, VOLUME_HEIGHT, VOLUME_WIDTH, 3), dtype=np.uint8)

    for depth, full_result, stage_result, image in zip(depths, full_results, stage_results, images):
        full_boxes = full_result.boxes
        stage_boxes = stage_result.boxes
        plant_idx = int(plant_idx)

        # Determine which box to use based on confidence and availability
        if full_boxes.xyxy.shape[0] > plant_idx and stage_boxes.xyxy.shape[0] > plant_idx:
            full_box, full_conf = full_boxes.xyxy[plant_idx], full_boxes.conf[plant_idx]
            stage_box, stage_conf = stage_boxes.xyxy[plant_idx], stage_boxes.conf[plant_idx]
            boxes = full_box if full_conf > stage_conf else stage_box
        elif full_boxes.xyxy.shape[0] > plant_idx:
            boxes = full_boxes.xyxy[plant_idx]
        elif stage_boxes.xyxy.shape[0] > plant_idx:
            boxes = stage_boxes.xyxy[plant_idx]
        else:
            continue

        # Crop the image based on the selected box
        x1, y1, x2, y2 = map(int, boxes)
        image_cropped = np.asarray(image)[y1:y2, x1:x2]

        if verbose:
            print(f"Depth: {depth}")
            display(Image.fromarray(image_cropped).convert('RGB'))

        # Resize and convert the cropped image
        image_cropped = Image.fromarray(image_cropped).resize((VOLUME_WIDTH, VOLUME_HEIGHT)).convert('RGB')
        image_cropped = np.asarray(image_cropped)

        # Store the cropped image in the volumes array
        volumes[depth - START_BEGIN] = image_cropped

    return volumes

In [ ]:
images, depths = load_images(TRAIN_DATA, sample['FolderName'], sample['Side'], START_BEGIN, END_BEGIN)
volumes = detect_image(images, depths, sample['Stage'], sample['PlantNumber'], verbose=True)

Depth: 8


Depth: 9


Depth: 10


Depth: 11


In [ ]:
volumes = {}
categoricals = []
targets = []
for row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
  sample = row[1]
  images, depths = load_images(TRAIN_DATA, sample['FolderName'], sample['Side'], START_BEGIN, END_BEGIN)
  volumes[sample['ID']] = detect_image(images, depths, sample['Stage'], sample['PlantNumber'])
  categoricals.append((sample['Stage'], sample['Side'], sample['Genotype'], sample['ID']))
  targets.append(sample['RootVolume'])

meta_df = pd.DataFrame(categoricals, columns=['Stage', 'Side', 'Genotype', 'ID'])
meta_df['RootVolume'] = targets

  0%|          | 0/386 [00:00<?, ?it/s]

In [ ]:
meta_df.head()

,Stage,Side,Genotype,ID,RootVolume
0,Early,L,IITA-TMS-IBA000070,ID_826322_Lbmaya,0.9
1,Early,L,IBA154810,ID_718181_Bslpwx,1.5
2,Early,R,IBA980581,ID_465762_L1n61d,2.7
3,Early,R,IITA-TMS-IBA000070,ID_626872_Pbmx2e,2.6
4,Early,R,IBA980581,ID_518846_Opko8c,2.7


In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),        # Randomly flip image horizontally with 50% probability.
    transforms.RandomRotation(degrees=15),           # Rotate the image by a random angle up to ±15°.
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, saturation, and hue.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize as per ImageNet statistics
                         std=[0.229, 0.224, 0.225])
])

# Transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, meta_df, volumes, target_col='RootVolume', transform=None):
    self.transform = transform
    self.target_col = target_col
    self.meta_df = meta_df
    self.volumes = volumes

  def __len__(self):
    return self.meta_df.shape[0]

  def __getitem__(self, idx):
    sample = self.meta_df.iloc[idx]
    volume = self.volumes[sample['ID']]
    if self.transform:
      volume_aug = []
      for frame in volume:
        volume_aug.append(self.transform(frame))
      volume = torch.stack(volume_aug)

    if self.target_col is None:
      return volume

    target = sample[self.target_col]
    return {'pixel_values': volume, 'labels': target}

In [ ]:
class Mixup:
  def  __init__(self, alpha=0.2):
    self.alpha = alpha

  def __call__(self, batched):
    pixel_values = [batch['pixel_values'] for batch in batched]
    labels = [batch['labels'] for batch in batched]
    lam = np.random.beta(self.alpha, self.alpha, pixel_values[0].shape[0])
    shuffled_pixel_values = pixel_values.copy()

    indices = torch.randperm(pixel_values[0].shape[0])


In [ ]:
meta_df.shape

(386, 5)

In [ ]:
ds = Dataset(meta_df, volumes, transform=train_transform)
dataloader = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=True)

In [ ]:
v = next(iter(dataloader))

In [ ]:
v['pixel_values'].shape

torch.Size([4, 53, 3, 64, 512])

In [ ]:
from transformers import PretrainedConfig, PreTrainedModel
from torch import nn

In [ ]:
class RootEstimatorConfig(PretrainedConfig):
    config_class = PretrainedConfig
    def __init__(self, projection_dim=256, feature_extractor='resnet18', num_layers=1, dropout=0.0, **kwargs):
        super().__init__(**kwargs)
        self.projection_dim = projection_dim
        self.feature_extractor = feature_extractor
        self.num_layers = num_layers
        self.dropout = dropout

class RootEstimator(PreTrainedModel):
    config_class = RootEstimatorConfig
    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.backbone = timm.create_model(self.config.feature_extractor, pretrained=True, in_chans=3)
        self.num_features = self.backbone.fc.in_features
        self.backbone.global_pool = nn.Identity()
        self.backbone.fc = nn.Identity()
        self.lstm = nn.LSTM(self.num_features, self.config.projection_dim, self.config.num_layers)
        self.dropout = nn.Dropout(self.config.dropout)
        self.head = nn.Linear(self.config.projection_dim, 1)

    def forward(self, pixel_values, labels=None):
        input = pixel_values
        input = input.float()
        batch, frame, width, height, channels = input.shape
        frame_idx = 0
        feature = self.backbone(input[:, frame_idx])
        feature = feature.view((batch, self.num_features, -1))
        feature = feature.permute(0, 2, 1)
        out, *state = self.lstm(feature)
        for frame_idx in range(1, frame):
            feature = self.backbone(input[:, frame_idx])
            feature = feature.view((batch, self.num_features, -1))
            feature = feature.permute(0, 2, 1)
            out, *state = self.lstm(feature, *state)
        out = self.dropout(out)
        out = out.mean(dim=1)
        out = self.head(out)
        out = out.squeeze(-1)
        if labels is not None:
            labels = labels.float()
            loss = nn.MSELoss()(out, labels)
        return {'loss': loss, 'logits': out}

In [ ]:
# ---------------------------
# Configuration
# ---------------------------
class RootEstimatorConfig(PretrainedConfig):
    config_class = PretrainedConfig

    def __init__(self, projection_dim=256, feature_extractor='resnet18', num_layers=1, dropout=0.0, **kwargs):
        """
        Args:
            projection_dim (int): Dimensionality used both for FPN output and LSTM input.
            feature_extractor (str): Backbone model name for timm.
            num_layers (int): Number of layers in the LSTM.
            dropout (float): Dropout probability applied before the final head.
        """
        super().__init__(**kwargs)
        self.projection_dim = projection_dim
        self.feature_extractor = feature_extractor
        self.num_layers = num_layers
        self.dropout = dropout

# ---------------------------
# Pyramid Feature Extractor (FPN)
# ---------------------------
class PyramidFeatureExtractor(nn.Module):
    """
    A simple FPN that uses lateral 1x1 convolutions and a top-down pathway to fuse
    multi-scale features into a single feature map.
    """
    def __init__(self, in_channels_list, out_channels):
        """
        Args:
            in_channels_list (list[int]): List of channel dimensions for each feature map.
            out_channels (int): The desired number of output channels.
        """
        super().__init__()
        self.lateral_convs = nn.ModuleList([
            nn.Conv2d(in_ch, out_channels, kernel_size=1)
            for in_ch in in_channels_list
        ])
        self.fpn_convs = nn.ModuleList([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
            for _ in in_channels_list
        ])

    def forward(self, features):
        """
        Args:
            features (list[Tensor]): List of feature maps from the backbone.
                                     Assumes features are ordered from high-resolution
                                     to low-resolution.
        Returns:
            Tensor: Fused feature map of shape (B, out_channels, H, W), where H and W
                    match the resolution of the highest-resolution feature.
        """
        # Apply lateral convolutions.
        laterals = [l_conv(f) for f, l_conv in zip(features, self.lateral_convs)]
        # Top-down pathway: upsample and add.
        for i in range(len(laterals) - 1, 0, -1):
            upsampled = nn.functional.interpolate(laterals[i],
                                                    size=laterals[i - 1].shape[-2:],
                                                    mode='nearest')
            laterals[i - 1] += upsampled
        # Apply 3x3 convolutions.
        outs = [fpn_conv(lat) for lat, fpn_conv in zip(laterals, self.fpn_convs)]
        # Upsample all feature maps to the resolution of the highest-resolution feature and fuse (sum).
        out_size = outs[0].shape[-2:]
        outs_upsampled = [outs[0]] + [
            nn.functional.interpolate(o, size=out_size, mode='nearest')
            for o in outs[1:]
        ]
        fused = torch.sum(outs_upsampled)
        return fused

# ---------------------------
# Root Estimator Model with Pyramid Features
# ---------------------------
class RootEstimator(PreTrainedModel):
    config_class = RootEstimatorConfig

    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        # Create the backbone with features_only=True to extract intermediate features.
        self.backbone = timm.create_model(
            config.feature_extractor,
            pretrained=True,
            in_chans=3,
            features_only=True,
        )
        # Get the list of number of channels for each feature map.
        in_channels_list = [f["num_chs"] for f in self.backbone.feature_info]
        # Define the pyramid (FPN) module to fuse multi-scale features.
        self.fpn = PyramidFeatureExtractor(in_channels_list, config.projection_dim)
        # Define an LSTM to capture temporal dynamics.
        self.lstm = nn.LSTM(
            input_size=config.projection_dim,
            hidden_size=config.projection_dim,
            num_layers=config.num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(config.dropout)
        # Final head for regression (output single value per example).
        self.head = nn.Linear(config.projection_dim, 1)

    def forward(self, pixel_values, labels=None):
        """
        Args:
            pixel_values (Tensor): Input video frames of shape (B, T, H, W, C).
                                   Expected to be channels-last (C=3). They will be
                                   converted to channels-first.
            labels (Tensor, optional): Target values for computing loss.
        Returns:
            dict: Dictionary with keys 'logits' (model output) and optionally 'loss'.
        """
        x = pixel_values.float()  # (B, T, C, H, W)
        batch_size, num_frames, C, H, W = x.shape

        # ---- Process All Frames in a Batch Efficiently ----
        # Merge batch and time dimensions for backbone processing.
        x_reshaped = x.view(batch_size * num_frames, C, H, W)  # (B*T, C, H, W)
        # Extract multi-scale features for all frames.
        features_list = self.backbone(x_reshaped)  # list of Tensors, each (B*T, ch, H_i, W_i)
        # Fuse pyramid features.
        fused = self.fpn(features_list)  # (B*T, projection_dim, H_fpn, W_fpn)
        # Global average pooling to obtain a single feature vector per frame.
        pooled = fused.mean(dim=(-2, -1))  # (B*T, projection_dim)
        # Reshape back to (B, T, projection_dim)
        features_seq = pooled.view(batch_size, num_frames, self.config.projection_dim)
        # -----------------------------------------------------

        # Pass the per-frame feature sequence through the LSTM.
        lstm_out, _ = self.lstm(features_seq)  # (B, T, projection_dim)
        # Temporal aggregation: mean pooling over time.
        aggregated = lstm_out.mean(dim=1)  # (B, projection_dim)
        aggregated = self.dropout(aggregated)
        logits = self.head(aggregated).squeeze(-1)  # (B,)

        loss = None
        if labels is not None:
            labels = labels.float()
            loss = nn.MSELoss()(logits, labels)
        return {'loss': loss, 'logits': logits}

In [ ]:
def empty_cache():
    import gc
    torch.cuda.empty_cache()
    torch._C._cuda_emptyCache()
    gc.collect()

In [ ]:
config = RootEstimatorConfig()
model = RootEstimator(config)

In [ ]:
from datetime import datetime
from transformers import TrainingArguments

args = TrainingArguments(
    run_name='Run',
    output_dir='./results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=100,
    warmup_steps=64,
    weight_decay=0.01,
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_steps=64,
    report_to='none',
    eval_strategy='epoch',
    metric_for_best_model='loss',
    save_total_limit=1,
    load_best_model_at_end=True,
    greater_is_better=False,
    save_strategy='epoch',
)

In [ ]:
df_train, df_valid = train_test_split(meta_df, test_size=0.2, random_state=42)

train_ds = Dataset(df_train, volumes, transform=train_transform)
valid_ds = Dataset(df_valid, volumes, transform=transform)

print(len(train_ds), len(valid_ds))

308 78


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds
)

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 848.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 326.12 MiB is free. Process 2929 has 14.42 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 15.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)